### ztf_sim_introduction

This notebook illustrates basic use of the `ztf_sim` modules.

In [1]:
# hack to get the path right
import sys
sys.path.append('..')

In [78]:
import ztf_sim
from astropy.time import Time
import pandas as pd
import numpy as np
import astropy.units as u
import pylab as plt
from matplotlib import animation
import astropy.coordinates as coords
from collections import Counter

First we'll generate a test field grid.  You only need to do this the first time you run the simulator.

In [ ]:
ztf_sim.fields.generate_test_field_grid()

Let's load the Fields object with the default field grid:

In [ ]:
f = ztf_sim.fields.Fields()

The raw fieldid and coordinates are stored as a pandas Dataframe in the `.fields` attribute:

In [ ]:
f.fields

Now let's calcuate their altitude and azimuth at a specific time using the astropy.time.Time object:

In [ ]:
f.alt_az(Time.now())

Demonstrate accessing fields by `fieldid`:

In [ ]:
w = f.fields['dec'] < 0.
f.fields[w].loc[853]

Calculating the overhead time (max of ha, dec, dome slews and readout time):

In [ ]:
f.overhead_time(853,Time.now())

In [130]:
f = ztf_sim.fields.Fields()
Exposure_time = 30*u.second
Night_length=9*u.h


time0 = Time('2015-09-10 20:00:00') + 7*u.h # First observatoins start + offset to Palomar
time = time0

f.fields = f.fields.join(pd.DataFrame(np.zeros(len(f.fields)),columns=['observed'])) #Counts how many times the field have been observed
f.fields = f.fields.join(pd.DataFrame(np.zeros(len(f.fields)),columns=['possibleToObserve'])) # Marks if it was possible to observe


def observe(f, nightStart):
    time=nightStart
    obsLog = {'time':np.array([]),'fieldid':np.array([])}
    
    firstField = True # Different treatment for the first field
    
    timeFirstObserve = time #The time at the begining of the first "cluster" - after some time (defined later) this cluster should be observed again
    
    while time < nightStart + Night_length:
        
        
        goodAltitude = f.alt_az(time)['alt'] > 20        
        shouldObserve = f.fields['observed'] < 1 # Observe fields which haven't been observed before
        good = goodAltitude & shouldObserve
        f.fields['possibleToObserve'][goodAltitude] = 1

        if not np.any(good): # If there are no possible fields for observation
            time += 60*u.s
            continue
            
        if firstField:
            better = good & (f.alt_az(time + 3*u.h)['alt'] > 20) & (f.alt_az(time + 4*u.h)['alt'] < 20) # Choose a field which will go down after 4hours
            if np.any(better): good = better
            fid = f.fields[good].iloc[0].name # pick the first one in the list
            obsLog['time']=np.append(obsLog['time'],(time - 7*u.h))
            obsLog['fieldid']=np.append(obsLog['fieldid'],fid)
            f.fields['observed'][fid]+=1
                               
            goodAltitude = f.alt_az(time)['alt'] > 20
            shouldObserve = f.fields['observed'] < 1
            good = goodAltitude & shouldObserve
            firstField = False
            
                        
        slewTime = f.overhead_time(fid,time)[good] # calculate slew time to all other possible fields
                
        fid = int(slewTime.idxmin()) # choose next field as the one which takes the shortest time to slew to
        time += Exposure_time + slewTime['overhead_time'][fid]*u.second # gives the time when starting to observe the current field
        obsLog['time']=np.append(obsLog['time'],(time - 7*u.h))
        obsLog['fieldid']=np.append(obsLog['fieldid'],fid)
        f.fields['observed'][fid]+=1
        
        if time - timeFirstObserve > 1*u.h: #Take another observation of the last "cluster"
            time, fid = secondObserve(fid, time, obsLog, f)
            timeFirstObserve = time
   
    return pd.DataFrame(obsLog)

def secondObserve(fid0, time, obsLog,f):
    index = f.fields[f.fields.observed ==1].index #All the fields which have been observed only once
    fields = obsLog['fieldid'][np.in1d(obsLog['fieldid'],index)].astype('int')
    for fid in fields: 
        
        if f.alt_az(time).alt.loc[fid]>20:            
            time += f.overhead_time(fid0,time).overhead_time[fid]*u.s + Exposure_time
            obsLog['time']=np.append(obsLog['time'],(time - 7*u.h))
            obsLog['fieldid']=np.append(obsLog['fieldid'],fid)
            f.fields['observed'][fid]+=1            
            fid0 = fid
    return time, fid
    
            
    
# First night
log = observe(f,time)
fieldsPossible = np.sum(f.fields['possibleToObserve'])
print fieldsPossible
fieldsObserved = np.sum(f.fields['observed'])
print fieldsObserved
meanTime = ((log.time.iloc[-1]-log.time.iloc[0] - (fieldsObserved-1)*Exposure_time)/(fieldsObserved-1)).to(u.second)
print meanTime


# Second night
time=time0+24*u.h
log = log.append(observe(f,time),ignore_index=True)
fieldsPossible = np.sum(f.fields['possibleToObserve'])
print fieldsPossible
fieldsObserved = np.sum(f.fields['observed'])
print fieldsObserved
meanTime = (2*Night_length.to(u.s)-fieldsObserved*Exposure_time)/(fieldsObserved-1)
print meanTime


562.0
827.0
13.2721201392 s
566.0
1026.0
33.1902439024 s


In [6]:
def plotFields(f):
    for dec in np.append(np.linspace(-90,90,10),0):
        ra=np.linspace(0, 360,1000)
        x,y = raDec2xy(ra,dec)
        plt.plot(x,y,'k')

    for ra in np.linspace(0,360,10):
        dec=np.linspace(-90, 90,1000)
        x,y = raDec2xy(ra,dec)
        plt.plot(x,y,'k')

    x,y = raDec2xy(f.fields['ra'],f.fields['dec'])
    plt.plot(x,y,'o',color=(.8,.8,.8))    
    #plt.show()

In [7]:
def raDec2xy(ra,dec):
    # Using Aitoff projections (from Wiki) returns x-y coordinates on a plane from RA and Dec
    theta = np.deg2rad(dec)
    phi = np.deg2rad(ra)-np.pi #the range is [-pi,pi]
    alpha=np.arccos(np.cos(theta)*np.cos(phi/2))
    x=2*np.cos(theta)*np.sin(phi/2)/np.sinc(alpha/np.pi) # The python's sinc is normalazid, hence the /pi
    y=np.sin(theta)/np.sinc(alpha/np.pi)
    return x,y

In [132]:
def showObservation(f,obsLog): # Takes the fields (f) and the observation log
    
    fig = plt.figure()
    ax = plt.axes(xlim=(-3.5, 3.5), ylim=(-2, 2))
    plotFields(f)
    line1, = ax.plot([], [],'.b')
    line2, = ax.plot([], [],'og')
    obsLimits, = ax.plot([],[],'xr')
    time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)
    
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        obsLimits.set_data([],[])
        time_text.set_text('')
        return line1,line2,obsLimits,time_text    

    def animate(i):
        
        counts = dict(Counter(obsLog.iloc[:i].fieldid))
        time = obsLog.time.iloc[i]

        # Fields which are observed for the first time
        fids1 = np.array(counts.keys())[np.array(counts.values()) == 1]
        ra = f.fields.ra[fids1]
        dec = f.fields.dec[fids1]
        
        x,y = raDec2xy(ra,dec)
        line1.set_data(x, y)
        
        # Fields which are observed for the second time
        fids2 = np.array(counts.keys())[np.array(counts.values()) == 2]
        ra = f.fields.ra[fids2]
        dec = f.fields.dec[fids2]
        
        x,y = raDec2xy(ra,dec)
        line2.set_data(x, y)
        
        """
        az = np.linspace(0,360,1000)
        alt = 50*np.ones(1000)
        altAzLim = coords.SkyCoord(az = az, alt = alt,frame='altaz', unit = 'deg', 
                           location = coords.EarthLocation(lat=coords.Latitude('33d21m26.35s'), 
                                                           lon=coords.Longitude('-116d51m32.04s'), 
                                                           height=1707.*u.m), obstime = obsLog.time[i])
        raDecLims = altAzLim.icrs
        x,y = raDec2xy(raDecLims.ra.value, raDecLims.dec.value) # passed dimensionless numbers in degrees
        """
        
        # Fields which are at good altitiude and possible to observe
        goodAltitude = f.alt_az(time + 7*u.h)['alt'] > 20
        x,y = raDec2xy(f.fields[goodAltitude].ra,f.fields[goodAltitude].dec)        
        obsLimits.set_data(x,y)
        
        # Displays the time
        time_text.set_text(time.value)
        
     
        return line1,line2,obsLimits,time_text
    
    frames = len(obsLog)
    anim = animation.FuncAnimation(fig, animate, init_func=init, frames=frames, blit=True)
    
    #plt.rcParams['animation.ffmpeg_path'] = '/home/danny/ffmpeg-2.8'
    #FFwriter = animation.FFMpegWriter()
    
    anim.save('basic_animation.mp4', writer = 'avconv', fps=5, dpi = 100) # Takes a lot of time, may be sensitive to writer on different platforms
    #plt.show()
    plt.close()

showObservation(f,log)


In [80]:
index = f.fields[f.fields.observed ==1].index

In [90]:
index

Int64Index([ 61,  64,  90,  93, 124, 128, 129, 162, 168, 169, 204, 212, 213,
            250, 260, 261, 298, 309, 310, 347, 348, 361, 362, 398, 413, 414,
            416, 450, 466, 467, 501, 519, 520, 552, 571, 572, 601, 622, 623,
            648, 671, 672, 673, 674, 675, 684, 685, 686, 687, 688, 691, 721,
            722, 726, 727],
           dtype='int64', name=u'fieldid')

In [111]:
log.fieldid.values

array([  40.,   20.,   39., ...,  348.,  450.,  552.])

In [ ]:
az = np.linspace(0,360,1000)
alt = 20*np.ones(1000)
altAzLim = coords.SkyCoord(az = az, alt = alt,frame='altaz', unit = 'deg', 
                           location = coords.EarthLocation(lat=coords.Latitude('33d21m26.35s'), 
                                                           lon=coords.Longitude('-116d51m32.04s'), 
                                                           height=1707.*u.m), obstime = Time.now())
raDecLims = altAzLim.icrs

In [115]:
log[np.in1d(log.fieldid.values,index)].fieldid.values

array([ 684.,  685.,  686.,  687.,  688.,   93.,  128.,  129.,  169.,
        168.,  212.,  213.,  310.,  309.,  260.,  261.,  361.,  362.,
        414.,  413.,  466.,  467.,  520.,  519.,  571.,  572.,  623.,
        622.,  671.,  672.,  675.,  674.,  673.,  416.,  721.,  722.,
         64.,  726.,  727.,  347.,  691.,  398.,   61.,  162.,  204.,
        124.,   90.,  601.,  648.,  501.,  298.,  250.,  348.,  450.,  552.])

In [ ]:
a=Counter(log[:30].fieldid)
print a

In [ ]:
b=dict(a)
print b

In [ ]:
np.array(b.keys())[np.array(b.values())==2]

In [ ]:
b.keys()[5:9]

In [ ]:
a.iloc[-1]

In [ ]:
a = {'x':np.array([4,5,6]), 'y':np.array([8,9,10])}

In [ ]:
a['x'][a['y']>z]

In [ ]:
np.any

In [ ]:
a['y']<10

In [ ]:
f.fields[f.fields.dec<3].index[7]

In [ ]:
f.fields.loc[440]
